<a href="https://colab.research.google.com/github/JEETMANISHBOBDE/Image-Restoration/blob/main/Image_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# ✅ Verify the manually uploaded image path
file_path = "/content/baby.png"  # Change the filename if different

# ✅ Check if the file exists
if os.path.exists(file_path):
    print("File found:", file_path)

    # Load and preprocess the image
    img = cv2.imread(file_path)

    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))  # Resize to match the model input shape